<div style="border-radius:10px;
            border: #900c3f solid;
            background-color:#fff6f9;
            font-size:110%;
            letter-spacing:0.5px;
            text-align: center">

<center><h1 style="padding: 25px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
Pycaret in Regression case</h1></center>
<center><h3 style="padding-bottom: 25px; color:#900c3f; font-weight: bold; font-family: Cursive">
(House price prediction🏡🏤⛪)</h3></center>     

</div>

<div style="padding: 5px 0px; font-family: Cursive; font-size:17px; background-color:#fff6f9;padding: 25px 10px">
Following the review of the Pycaret library, in this notebook, we will examine a regression case.<br><br>
✔<i style = "font-size:16px;">(In the <a href="https://www.kaggle.com/arezoodahesh/heart-failure-prediction-with-pycaret"> Heart Failure Prediction | with Pycaret </a>, I modeled a classification case with the help of Pycaret)</i></div>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install -U dataprep

In [ ]:
!pip install pycaret[full]

# Import Libraries

In [ ]:
import pandas as pd
from pycaret.regression import *
from dataprep.eda import *

import warnings
warnings.filterwarnings("ignore")

# Data Understanding

In [ ]:
train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
test = pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")

In [ ]:
train.describe().T

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #900c3f;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
EDA with DataPrep</h3></center>
</div>

In [ ]:
plot_missing(train, display=["Bar Chart"])

In [ ]:
plot(train)

# Preprocessing and Modeling with Pycaret

In [ ]:
train = pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")

In [ ]:
categorical_feature = train.dtypes==object
final_categorical_feature = train.columns[categorical_feature].tolist()
#------------------------------------
categorical_feature_test = test.dtypes==object
final_categorical_feature_test = test.columns[categorical_feature_test].tolist()

# ********************************************************************

numeric_feature = train.dtypes!=object
final_numeric_feature = train.columns[numeric_feature].tolist()
final_numeric_feature = final_numeric_feature.remove('SalePrice')
#---------------------------------------------------------------------
numeric_feature_test = test.dtypes!=object
final_numeric_feature_test = test.columns[numeric_feature_test].tolist()

In [ ]:
setup_train = setup(data = train, target = 'SalePrice', session_id=100, silent=True,
                    ignore_features=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 
                                     'FireplaceQu', 'LotFrontage'], 
                    numeric_features= final_numeric_feature,
                    categorical_features= final_categorical_feature,
                    numeric_imputation='median',
                    remove_outliers= True, 
                    remove_multicollinearity= True, multicollinearity_threshold=0.80,
                    normalize = True) 

In [ ]:
top_3 = compare_models(n_select=3, sort='RMSE', exclude =['huber','dummy', 'lar'])

In [ ]:
top_3

# The top 3 models and Tunning them

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #900c3f;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
Bayesian Ridge</h3></center>
</div>

In [ ]:
BR = create_model('br')

In [ ]:
BR_tuned = tune_model(BR) 

In [ ]:
predict_model(BR_tuned);

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #900c3f;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
Ridge Regression</h3></center>
</div>

In [ ]:
Ridge = create_model('ridge')

In [ ]:
Ridge_tuned = tune_model(Ridge) 

In [ ]:
predict_model(Ridge_tuned);

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #900c3f;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
CatBoost Regressor</h3></center>
</div>

In [ ]:
CB = create_model('catboost')

In [ ]:
CB_tuned = tune_model(CB) 

In [ ]:
predict_model(CB_tuned);

# Blending top 3 models

In [ ]:
blend_model = blend_models(estimator_list = [BR_tuned, Ridge_tuned, CB_tuned])

In [ ]:
blend_model_tuned = tune_model(blend_model) 

In [ ]:
predict_model(blend_model_tuned);

# Stacking top 3 models

In [ ]:
estimators =[BR_tuned, Ridge_tuned, CB_tuned]
stack_model = stack_models(estimators, meta_model= BR_tuned)

In [ ]:
stack_models_tuned = tune_model(stack_model) 

In [ ]:
predict_model(stack_models_tuned);

# Final Model

<div style="padding: 5px 0px; font-family: Cursive; font-size:16px; background-color:#fff6f9;padding: 25px 10px">
The best model is stack_models_tuned</div>

In [ ]:
plot_model(stack_models_tuned, plot = 'residuals')

In [ ]:
plot_model(stack_models_tuned, plot = 'error')

In [ ]:
plot_model(stack_models_tuned, plot = 'learning')

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #900c3f;
            letter-spacing:0.5px;">

<center><h3 style="padding: 5px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
Finalize Model</h3></center>
</div>

In [ ]:
final_stack_models = finalize_model(stack_models_tuned)
print(final_stack_models)
print("\n*******************************\n")
predict_model(final_stack_models);

<div style="padding: 10px; font-family: Cursive; border: solid 2px #900c3f;
            font-size:15.5px;padding: 25px 10px; border-radius:8px;">
<p>🧐 finalize_model() :</p>

<p style="padding:0 30px; font-style:italic; color:#900c3f; text-align:justify">Model finalization is the last step in the experiment. A normal machine learning workflow in PyCaret starts with setup(), followed by comparing all models using compare_models() and shortlisting a few candidate models (based on the metric of interest) to perform several modeling techniques such as hyperparameter tuning, ensembling, stacking etc. This workflow will eventually lead you to the best model for use in making predictions on new and unseen data. The finalize_model() function fits the model onto the complete dataset including the test/hold-out sample (30% in this case). The purpose of this function is to train the model on the complete dataset before it is deployed in production.<br><br>
ref : <a href="https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb">Regression Tutorial Level Beginner</a></p>


</div>

# Submission

In [ ]:
pred_stack_models = predict_model(final_stack_models, data=test)
test['SalePrice'] = pred_stack_models['Label']
output_stack_models = pd.DataFrame({'Id': test.Id, 'SalePrice': test.SalePrice})
output_stack_models.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")
output_stack_models.head()

# References

* https://pycaret.gitbook.io/docs/get-started/quickstart#regression
* https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Beginner%20-%20REG101.ipynb
* https://github.com/pycaret/pycaret/blob/master/tutorials/Regression%20Tutorial%20Level%20Intermediate%20-%20REG102.ipynb
* https://pycaret.gitbook.io/docs/get-started/functions/analyze#customize-the-plot

<div style="border-radius:10px;
            background-color:#ffffff;
            border-style: solid;
            border-color: #900c3f;
            letter-spacing:0.5px;">

<center><h4 style="padding: 5px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
    Thanks for your attention and for reviewing my notebook.🙌 <br><br>Please write your comments for me.📝</h4></center>
<center><h4 style="padding: 5px 0px; color:#900c3f; font-weight: bold; font-family: Cursive">
If you liked my work and found it useful, please upvote. Thank you🙏</h4></center>
</div>